In [1]:
import torch
import torchvision
from torch import nn, optim, utils
from torchvision import transforms, datasets
from torchsummary import summary

import matplotlib.pyplot as plt
from PIL import Image

print("torch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch version: 2.2.1+cu121
torchvision version: 0.17.1+cu121
CUDA available: True


In [2]:
import pickle
import numpy as np

with open("metric_vals.pkl", "rb") as file:
    data = pickle.load(file)

In [3]:
data

([3.737566876091106],
 [0.20234],
 [3.31071610648792],
 [0.1975],
 [0.20226914122494352],
 [0.19750000000000004],
 [0.1719913154676058])

In [ ]:
def plot_vals(vals, title, y_label):

    x = np.arange(len(vals))

    # Plotting the line chart
    plt.plot(x, y, marker='o', linestyle='-')

    # Adding labels and title
    plt.xlabel("epoch")
    plt.ylabel(y_label)
    plt.title(title)

    # Displaying the plot
    plt.grid(True) # Add grid lines
    plt.show()

In [3]:
# Load in data and inspect

root = "/home/hoawenlo/Programming/Python/pytorch_practice/img_class_with_dvc/cifar-100-python"

train_cifar100 = datasets.CIFAR100(root=root, train=True)
val_cifar100 = datasets.CIFAR100(root=root, train=False, transform=transforms.ToTensor())

In [7]:
train_cifar100

Dataset CIFAR100
    Number of datapoints: 50000
    Root location: /home/hoawenlo/Programming/Python/pytorch_practice/img_class_with_dvc/cifar-100-python
    Split: Train

In [4]:
val_cifar100

Dataset CIFAR100
    Number of datapoints: 10000
    Root location: /home/hoawenlo/Programming/Python/pytorch_practice/img_class_with_dvc/cifar-100-python
    Split: Test
    StandardTransform
Transform: ToTensor()

In [13]:
import numpy as np
img, label = train_cifar100[0]
np.array(img)

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [195, 205, 193],
        [212, 224, 204],
        [182, 194, 167]],

       [[255, 255, 255],
        [254, 254, 254],
        [254, 254, 254],
        ...,
        [170, 176, 150],
        [161, 168, 130],
        [146, 154, 113]],

       [[255, 255, 255],
        [254, 254, 254],
        [255, 255, 255],
        ...,
        [189, 199, 169],
        [166, 178, 130],
        [121, 133,  87]],

       ...,

       [[148, 185,  79],
        [142, 182,  57],
        [140, 179,  60],
        ...,
        [ 30,  17,   1],
        [ 65,  62,  15],
        [ 76,  77,  20]],

       [[122, 157,  66],
        [120, 155,  58],
        [126, 160,  71],
        ...,
        [ 22,  16,   3],
        [ 97, 112,  56],
        [141, 161,  87]],

       [[ 87, 122,  41],
        [ 88, 122,  39],
        [101, 134,  56],
        ...,
        [ 34,  36,  10],
        [105, 133,  59],
        [138, 173,  79]]

In [6]:
train_imgs = torch.stack([img[0] for img in train_cifar100])
# val_imgs = torch.stack([img[0] for img in val_cifar100])
# train_imgs.shape, val_imgs.shape
train_imgs.shape

TypeError: expected Tensor as element 0 in argument 0, but got Image

In [7]:
train_labels = torch.Tensor([label[1] for label in train_cifar100])
val_labels = torch.Tensor([label[1] for label in val_cifar100])
train_labels.shape, val_labels.shape

(torch.Size([50000]), torch.Size([10000]))

In [8]:
# Normalise data

def calc_mean_std(input_data):
    """Calculate the mean and standard deviation across each channel.
    
    Args:
        input_data: The input data. Is a torch tensor.
        
    Returns:
        The mean and standard deviation of each colour channel of the dataset.
        The values are torch tensors of shape (3,)."""
    
    batch_size = input_data.shape[0]
    mean = torch.mean(input_data.view(batch_size, 3, -1), dim=(0, 2))
    std = torch.std(input_data.view(batch_size, 3, -1), dim=(0, 2))
    
    return mean, std

def apply_transformation(input_data):
    """Apply transformations to the data.
    
    Args:
        input_data: The input data. Is a torch tensor.
        
    Returns:
        Data with transformations applied. Since only normalisation has been applied
        the shape of the data will be the same."""
    
    mean, std = calc_mean_std(input_data)
    
    transformations = transforms.Compose([
        transforms.Normalize(mean, std)
    ])

    transformed_data = transformations(input_data)
    
    return transformed_data

In [9]:
normalised_train_imgs = apply_transformation(train_imgs)
normalised_val_imgs = apply_transformation(val_imgs)

normalised_train_imgs.shape, normalised_val_imgs.shape

(torch.Size([50000, 3, 32, 32]), torch.Size([10000, 3, 32, 32]))

In [11]:
# Move data to GPU

normalised_train_imgs = normalised_train_imgs.to(device)
normalised_val_imgs = normalised_val_imgs.to(device)

print("Train images is CUDA: ", normalised_train_imgs.is_cuda)
print("Val images is CUDA: ", normalised_val_imgs.is_cuda)

Train images is CUDA:  True
Val images is CUDA:  True


In [42]:
# Move labels to GPU

train_labels = train_labels.to(device).to(torch.long)
val_labels = val_labels.to(device).to(torch.long)

print("Train images is CUDA: ", train_labels.is_cuda)
print("Val images is CUDA: ", val_labels.is_cuda)

Train images is CUDA:  True
Val images is CUDA:  True


In [22]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        return x
    
class FullyConnectedNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8 * 8 * 8, 256)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 100)

    def forward(self, x):
        x = self.act1(self.fc1(x))
        x = self.act2(self.fc2(x))
        x = self.fc3(x)

        return x
    
class Model(nn.Module):

    def __init__(self):
        super().__init__()

        self.convnet = ConvNet()
        self.fcnet = FullyConnectedNet()
    
    def forward(self, x):

        x = self.convnet(x)
        x = x.view(-1, 512)
        x = self.fcnet(x)
        return x


In [20]:
model = Model()

model.to(device)

print("Model is on cuda: ", next(model.parameters()).is_cuda)

Model is on cuda:  True


In [24]:
# View model summary
summary(model, input_size=train_imgs.shape[1:])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
              ReLU-2           [-1, 16, 32, 32]               0
         MaxPool2d-3           [-1, 16, 16, 16]               0
            Conv2d-4            [-1, 8, 16, 16]           1,160
              ReLU-5            [-1, 8, 16, 16]               0
         MaxPool2d-6              [-1, 8, 8, 8]               0
           ConvNet-7              [-1, 8, 8, 8]               0
              ReLU-8                  [-1, 512]               0
            Linear-9                  [-1, 256]         131,328
             ReLU-10                  [-1, 256]               0
           Linear-11                  [-1, 128]          32,896
           Linear-12                  [-1, 100]          12,900
FullyConnectedNet-13                  [-1, 100]               0
Total params: 178,732
Trainable params:

In [91]:
import datetime

def train(optimiser, model, loss_fn, train_loader):
    """Train the model.
    
    Args:
        optimiser: The optimiser used.
        model: The trained model used to calculate the accuracy.
        loss_fn: The loss function.
        train_loader: The train dataset dataloader.
        val_loader: The validation dataset dataloader.
        
    Returns:
        Training and validation loss."""

    model.train()
    train_loss = 0.0
    for imgs, labels in train_loader:

        # Forward pass
        outputs = model(imgs)

        # Calculate loss
        loss = loss_fn(outputs, labels)

        # Reset grad vals.
        optimiser.zero_grad()

        # Perform back propagation
        loss.backward()

        # Update weights
        optimiser.step()

        # Sum the losses over each iteration of the epoch and conver to python number
        # to avoid Pytorch autograd being applied to it.
        train_loss += loss.item()
    train_loss /= len(train_loader)

    return train_loss

def validate(model, loss_fn, val_loader):
    """Calculate validation loss.
    
    Args:
        model: The trained model used to calculate the accuracy.
        loss_fn: The loss function.
        val_loader: The validation dataloader.
        
    Returns:
        Returns validation loss value."""
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, labels in val_loader:
            
            # Forward pass
            outputs = model(imgs)

            # Calculate loss
            loss = loss_fn(outputs, labels)

            # Calculate validation loss
            val_loss += loss.item()
    val_loss /= len(val_loader)
    return val_loss


def accuracy(model, data_loader):
    """Calculate the accuracy of the model.
    
    Args:
        model: The trained model used to calculate the accuracy.
        data_loader: The dataset dataloader.
        
    Returns:
        Accuracy of the model with the particular dataloader."""

    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in data_loader:
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            print(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total

    return accuracy



def training_loop(n_epochs, optimiser, model, loss_fn, train_loader, val_loader):
    """Run a training loop for model for n number of epochs. 
    Load in the data with dataloader then perform a forward pass.
    Calculate the loss, perform back propagation then update the parameters.
    Apply the same with validation dataset.
    
    Args:
        n_epochs: Number of epochs, is an integer.
        optimiser: The chosen optimiser such as SGD or Adam.
        model: The input model to perform the training with.
        loss_fn: The loss function.
        train_loader: The dataloader for the training data.
        val_loader: The dataloader for the validation data.
        
    Returns:
        None"""
    
    for epoch in range(1, n_epochs + 1):
        train_loss, val_loss = train(optimiser, model, loss_fn, train_loader, val_loader)
        train_acc = validate(model, train_loader)
        val_acc = validate(model, val_loader)

        # if epoch == 1 or epoch % 10 == 0:
        print(f"{datetime.datetime.now()}, Epoch {epoch}, Training loss {train_loss}, Validation loss {val_loss}, Training Accuracy {train_acc}, Validation Accuracy {val_acc}")


    

In [92]:
class CustomDataset(utils.data.Dataset):
    """Custom PyTorch dataset to feed in dataloader."""
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get the data and label at specified index.

        img = self.data[idx]
        label = self.labels[idx]

        return img, label

model = Model().to(device)

# Create custom dataset

train_data = CustomDataset(normalised_train_imgs, train_labels)
val_data = CustomDataset(normalised_val_imgs, val_labels)

# Setup data loader

train_dataloader = utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_dataloader = utils.data.DataLoader(val_data, batch_size=32, shuffle=True)

# Setup loss and optimiser

loss = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=0.01)

training_loop(
    n_epochs = 1,
    optimiser=optimiser,
    model=model,
    loss_fn=loss,
    train_loader=train_dataloader,
    val_loader=val_dataloader
)

tensor([92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92,
        92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
       device='cuda:0')
tensor([92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92,
        92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
       device='cuda:0')
tensor([92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92,
        92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
       device='cuda:0')
tensor([92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92,
        92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
       device='cuda:0')
tensor([92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92,
        92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
       device='cuda:0')
tensor([92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92,
        92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92],
       dev

In [80]:
train_labels

tensor([19, 29,  0,  ...,  3,  7, 73], device='cuda:0')